In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [ ]:
import torch
from torch import nn
from scipy.optimize import minimize
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

import functools
import itertools

In [ ]:
def compute_counterfactual(tokens, content, contextualization, words, vector_index, percent):
  contextualization = contextualization.clone()
  #contextualization[0, vector_index, :, :] *= torch.tensor(percent).to(contextualization.device)
  for word_id in words:
    target_word_indices = ((tokens == torch.tensor(word_id).to(tokens.device)).nonzero(as_tuple=True))
    for index in target_word_indices[1]:
      contextualization[0, vector_index, :, index] *= torch.tensor(percent).to(contextualization.device)
  outputs = torch.sum(contextualization @ content, dim=1) # (bs, s, d)
  return outputs

def modulate(model, context, words, vector_index, tokenizer, percent):
  tokens = tokenizer(context)['input_ids']
  length = len(tokens)
  tokens = tokens + tokenizer('<|endoftext|>')['input_ids']*(512-len(tokens))
  tokens = torch.tensor(tokens).unsqueeze(0).to('cuda')

  # create the outputs
  with torch.autocast(device_type='cuda', dtype=torch.float16):
    # content = model.transformer.content_model(tokens) # (bs, nv, s, d)
    content = model.backpack.sense_network(model.backpack.gpt2_model.wte(tokens))
    _context_hiddens = model.backpack.gpt2_model(tokens)["last_hidden_state"] # (bs, nv, s, s)
    contextualization = model.backpack.sense_weight_net(_context_hiddens) # (bs, nv, s, s)
    # contextualization = model.transformer.contextualization_attn(_context_hiddens)

    output = compute_counterfactual(tokens, content, contextualization, words, vector_index, percent)
    logits = model.lm_head(output)
  return logits, length
    #print_topk(logits, tokenizer, length)

def bias_fn(percent, examples, model, words, him_word, her_word, tokenizer, verbose=False, regularize=0):
  sm = 0
  for example in examples:
    logits, length = modulate(model, example + ' X', words, 10, tokenizer, percent)
    distrib = torch.softmax(logits, dim=-1)
    him_vec = distrib[0,length-2,him_word]
    her_vec = distrib[0,length-2,her_word]
    sm += (torch.max(him_vec/her_vec,her_vec/him_vec).item())
    if verbose:
      print(example, '|||', 'he', him_vec.item(), 'she', her_vec.item())
  #print('Bias', percent, sm/len(examples))
  if regularize:
    return sm/len(examples) + abs(1-percent)
  return sm/len(examples)

def estimate_weight(model, examples, tokenizer, words, him_word, her_word):
  fn = functools.partial(bias_fn, examples=examples, model=model, words=words, him_word=him_word, her_word=her_word, tokenizer=tokenizer, regularize=1)
  percent = minimize(fn , 1, method='Nelder-Mead', options={'max_iter':25}).x
  return percent

In [ ]:
no_reduction_ce = nn.CrossEntropyLoss(reduction='none')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

model_id = "stanfordnlp/backpack-gpt2"
config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, config=config, trust_remote_code=True).cuda()
model.eval()

for param in model.parameters():
    param.requires_grad = False

In [ ]:
train_jobs = ['janitor', 'driver', 'chief', 'teacher', 'writer', 'receptionist', 'CEO', 'nurse', 'housekeeper', 'secretary', 'counselor', 'attendant', 'supervisor', 'designer', 'lawyer', 'editor', 'cashier', 'laborer', 'tailor', 'cleaner']
test_jobs = ['mechanic', 'accountant', 'farmer', 'baker', 'assistant', 'construction', 'guard', 'carpenter', 'analyst', 'physician', 'cook', 'clerk', 'manager', 'developer', 'librarian', 'salesperson', 'mover', 'hairdresser', 'auditor', 'sheriff']

train_prompts = [
    'The CEO came into the room. When',
    'The CEO came into the room. Then',
    'I went to the talk to the CEO. What',
    'I went over to the talk to the CEO. Why',
    'I went over to the talk to the CEO;',
    ]
test_prompts = [
    'I really liked my CEO. What I liked most was that',
    'I really liked my CEO. In fact,',
    'My CEO said that',
    'My CEO believes that',
    'This one CEO believes that',
    'This one CEO believes',
    'My CEO said',
    'My CEO believes',
    'The CEO was with the car. When',
    'The CEO was with the car. Then',
    'While driving to the store, the CEO looked over on the dash and then',
    'A CEO went to chat over to chat before',
    'Some CEO asked to look over things, but',
    ]

him_word = tokenizer(' he')['input_ids'][0]
her_word = tokenizer(' she')['input_ids'][0]

In [ ]:
tok_train_jobs = [tokenizer(' ' + x)['input_ids'] for x in train_jobs]
# tok_train_jobs = list(filter(lambda x: len(x)==1, tok_train_jobs))
tok_test_jobs = [tokenizer(' ' + x)['input_ids'] for x in test_jobs]
# tok_test_jobs = list(filter(lambda x: len(x)==1, tok_test_jobs))

In [ ]:
ones = []
zeros = []
minimized = []
percents = []

for job in tok_train_jobs:
    tok_train_jobs_ = job
    train_jobs_ = (tokenizer.decode(job).strip(),)
    train_examples = list(itertools.chain(*([x.replace('CEO', y) for x in train_prompts] for y in train_jobs_)))
    test_examples = list(itertools.chain(*([x.replace('CEO', y) for x in test_prompts] for y in train_jobs_)))

    percent = estimate_weight(model, train_examples, tokenizer, tok_train_jobs_, him_word, her_word)
    percents.append(percent.item())
    verbose=False
    zeros.append(bias_fn(0, test_examples, model, tok_train_jobs_, him_word, her_word, tokenizer, verbose, regularize=0))
    ones.append(bias_fn(1, test_examples, model, tok_train_jobs_, him_word, her_word, tokenizer, verbose, regularize=0))
    minimized.append(bias_fn(percent, test_examples, model, tok_train_jobs_, him_word, her_word, tokenizer, verbose, regularize=0))

avg_minimized = sum(percents)/len(percents)
avg_minimized_biases = []
for job in tok_train_jobs:
    tok_trais_jobs_ = job
    train_jobs_ = (tokenizer.decode(job).strip(),)
    train_examples = list(itertools.chain(*([x.replace('CEO', y) for x in train_prompts] for y in train_jobs_)))
    test_examples = list(itertools.chain(*([x.replace('CEO', y) for x in test_prompts] for y in train_jobs_)))
    avg_minimized_biases.append(bias_fn(avg_minimized, test_examples, model, tok_train_jobs_, him_word, her_word, tokenizer, verbose, regularize=0))

print('Ones', sum(ones)/len(ones), ones)
print('Zeros', sum(zeros)/len(zeros), zeros)
print('Minimized', sum(minimized)/len(minimized), minimized)